In [ ]:
# 1. Self-attention by hand
# 2. Self-attention block in pytorch
# 3. GPT, piece-by-piece
# 4. GPU goes rrrr!

# Original code from https://github.com/karpathy/minGPT/tree/master/mingpt

### Step 1: Self-attention by hand

In [1]:
import torch
import math
import torch.nn.functional as F

In [ ]:
#  -- Write the scaled dot product self attention
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Scale the dot products
  # 4. Apply softmax to calculate attentions
  # 5. Weight values by attentions
  # 6. Compute attention weighted features

In [2]:
# Choose random values for the parameters -- sames values as on slide 12, but in pytorch format
# T = 4, C = 6, H = 3
X = torch.tensor([[2,0,0,0,2,1],[0,1,2,0,0,0],[0,0,1,1,0,1],[2,0,0,1,0,1]], dtype=float) # T x C
W_QT = torch.tensor([[1,0,0], [1,1,0], [0,0,1], [0,1,0], [0,0,1], [0,0,1]], dtype=float) # C x H
W_KT = torch.tensor([[0,0,1], [0,1,0], [1,0,0], [0,0,0], [0,0,0], [0,0,-1]], dtype=float) # C x H
W_VT = torch.tensor([[10,0,0], [0,0,10], [0,0,0], [0,10,0], [0,0,0], [0,0,0]], dtype=float) # C x H

In [34]:
# What does the second dimension of matrices Q and K correspond to?
Q = X @ W_QT  # torch.matmul(X, W_QT) # shape (T, H)
K = torch.matmul(X, W_KT)  # shape (T, H)
QKT = Q @ K.T # shape (T, T)
d_k = Q.shape[1]
QKT /= math.sqrt(d_k)

In [35]:
# compute the weighted attention matrix S
S = F.softmax(QKT, dim=1)  # shape (T, T)

In [37]:
# compute the self-attention matrix A
V = X @ W_VT  # shape (T, H)
A = S @ V

In [39]:
# Sanity check. This should return True.
torch.allclose(A.float(), torch.tensor([[10.30759701,  2.83283874,  4.59026201],
        [10.10551833,  2.97334971,  4.50027071],
        [15.03361159,  4.13169018,  2.10990693],
        [ 3.06082018,  1.53041009,  7.70438486]]))

True

### Step 2: Self-attention block in pytorch

In [40]:
import torch
import torch.nn as nn
from torch.functional import F

In [41]:
# do not modify this code

batch_size = 3 # B
block_size = 2 # T
n_embd = 3     # C

In [42]:
torch.set_printoptions(precision=8)

In [81]:
# Build a scaled self-attention head without masked attention and without dropout (i.e. just key, query and values)
# A matrix multiplication is implemented using the nn.Linear() operator with no bias.
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        

    def forward (self, x):
        B, T, C = x.shape
        K = self.key(x)
        Q = self.query(x)
        V = self.value(x)
        dot_product = Q @ K.transpose(-2, -1) * C**-0.5
        S = F.softmax(dot_product, dim=-1)
        out = S @ V
        return out

In [82]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
h = Head(2)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = h(x)
out

tensor([[[-0.06064763,  0.31713963],
         [-0.06091762,  0.31723329]],

        [[-0.16682129,  0.42035729],
         [-0.16693181,  0.41960484]],

        [[ 0.32534125,  0.85982299],
         [ 0.32709453,  0.85852110]]], grad_fn=<UnsafeViewBackward0>)

In [83]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.46728206,  0.03477207],
         [-0.47425330,  0.05069541]],
        [[-0.38198256,  0.02403205],
         [-0.39846635,  0.02506737]],
        [[-0.29631630,  0.12201238],
         [-0.30199534,  0.12650707]]]))

False

In [84]:
# Add weighted masked attention and dropout. Dropout comes after the softmax and before the multiplication with the value matrix.
# Copy the Head class from the previous exercise and expand upon it.

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        #self.dropout = nn.Dropout(.2)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # store a persistent buffer for the forward pass

    def forward (self, x):
        B, T, C = x.shape
        K = self.key(x)
        Q = self.query(x)
        V = self.value(x)
        dot_product = Q @ K.transpose(-2, -1) * C**-0.5
        dot_product = dot_product.masked_fill(self.tril == 0, float('-inf'))
        S = F.softmax(dot_product, dim=-1)
        #S = self.dropout(S)
        out = S @ V
        return out

In [72]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
h = Head(2)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = h(x)
out

tensor([[[-0.47424769, -0.20746115],
         [-0.59030378,  0.05762968]],

        [[-0.17730205,  0.01118639],
         [-0.49316812,  0.03102554]],

        [[-0.21627384,  0.03053588],
         [-0.37552872,  0.15657827]]], grad_fn=<UnsafeViewBackward0>)

In [73]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.37939820, -0.16596894],
         [-0.47425330,  0.05069541]],
        [[-0.14184165,  0.00894911],
         [-0.39846635,  0.02506737]],
        [[-0.17301908,  0.02442869],
         [-0.30199534,  0.12650707]]]))


False

In [74]:
# A multi-head attention module contains a list of heads and a linear projection layer.
# The heads are applied to the input and then concatenated along the last dimension, then
# the linear layer is applied. Look at the unit test below to determine the dimensions of
# the linear layer.

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size=head_size) for _ in range(num_heads)])

    def forward (self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return out

In [75]:
# do not modify
num_heads = 3
head_size = 2
n_embd = 6

In [76]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
sa = MultiHeadAttention(num_heads=3, head_size=head_size)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = sa(x)

In [77]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.03730504, -0.07006130, -0.27096999,  0.13144857, -0.45049590,
          -0.33217290],
         [-0.06818272, -0.04490501, -0.34806073,  0.15622401, -0.45459983,
          -0.33084857]],
        [[-0.08914752, -0.03846309, -0.36569631,  0.09802882, -0.39963537,
          -0.29225215],
         [-0.04541985,  0.01269679, -0.25225419,  0.08241771, -0.41533324,
          -0.30674040]],
        [[ 0.15234883, -0.08591781, -0.10099770,  0.19886394, -0.49236685,
          -0.43605998],
         [ 0.15418015, -0.01837257, -0.00573672,  0.14228639, -0.48172480,
          -0.40757987]]]))


False

In [85]:
# Add a classical feedforward module: linear -> ReLU -> linear
# The hidden dimension is four times bigger than the input dimension (see Section 3.3 of Attention is All You Need)
#
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd, bias=False),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd, bias=False)
            )

    def forward(self, x):
        out = self.network(x)
        return out

In [86]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
ff = FeedForward(n_embd)
torch.manual_seed(123) # do not remove this line
x = torch.rand((3,n_embd))
out = ff(x)
out

tensor([[ 0.18945211, -0.30948812, -0.16494906, -0.04409527,  0.17929617,
         -0.04507982],
        [ 0.14314917, -0.16567326, -0.12593222, -0.07840446,  0.13937519,
         -0.11273018],
        [ 0.17809552, -0.22925754, -0.10823976, -0.00855836,  0.13006634,
         -0.08191498]], grad_fn=<MmBackward0>)

In [87]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[-0.58034140,  0.04641046, -0.10707694,  0.21581653, -0.30361831,
         -0.07352637],
        [-0.48917407,  0.07879593, -0.15972012,  0.17862344, -0.37070659,
         -0.07852858],
        [-0.48530388,  0.09604470, -0.06524836,  0.16611034, -0.35499069,
         -0.08964306]]))

False

In [91]:
# Build a self-attention block
#
#   in -----> LayerNorm -------> multi-head attention -- + ----> LayerNorm -----> FeedForward --- + -----> out
#         |                                              |   |                                    |
#          ----------------------------------------------     ------------------------------------                       
#
# This architecture is slightly different from Attention is All You Need (or the UDL textbook):
# the layer norm comes before (not after) the attention or feed-forward
#
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.norm1 = nn.LayerNorm(n_embd)
        self.norm2 = nn.LayerNorm(n_embd)
        self.attn = MultiHeadAttention(n_head, head_size)
        self.ff = FeedForward(n_embd)

    def forward(self, x):
        out = self.norm1(x)
        out = self.attn(out)
        out = out + x
        out = self.norm2(out)
        out = self.ff(out)
        out = out + x
        return out

In [92]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
bk = Block(n_embd, num_heads)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size,block_size,n_embd))
out = bk(x)
out

tensor([[[ 0.35996270,  0.49915409,  0.03363548,  0.66058618,  0.11474703,
           0.94105983],
         [ 0.16384225,  0.05982038, -0.03269723,  0.63398296,  0.31917542,
           0.80163240]],

        [[ 0.05418029,  0.16768458,  0.07529587,  0.32104722,  0.13090128,
           1.02898717],
         [ 0.37756038,  0.65941703,  0.60925221,  0.53472948,  0.67063773,
           1.19953799]],

        [[ 0.41190907,  0.85719037,  0.27264974,  1.04809284,  0.90844464,
          -0.27160722],
         [ 1.11921954,  0.94238478,  0.73995113,  0.94792783,  1.13922715,
           0.16535005]]], grad_fn=<AddBackward0>)

In [93]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.05278997, -0.10863629, -0.09458938,  0.97590691, -0.55101192,
           0.57085067],
         [-0.16924502, -0.45394337, -0.25217158,  1.10904062, -0.34593600,
           0.41432184]],
        [[-0.41515028, -0.30126408, -0.11399293,  0.64651299, -0.51579159,
           0.57017863],
         [-0.02535054,  0.08704096,  0.66524690,  0.69768047,  0.05969021,
           0.69993609]],
        [[ 0.52881187,  0.34458166,  0.31130391,  1.11564195,  0.37998506,
          -0.02971917],
         [ 1.38496208,  0.60325992,  0.99346304,  0.38082033,  0.62151432,
           0.47973478]]]))

False

In [ ]:
## Step 3: Build a mini GPT
#
# - Start from the gpt-problem.py file
# - Add your Head, MultiHeadAttention, FeedForward and Block classes
# - Fill in the GPT class (__init__ and forward methods)
# - Train the network on CPU
# - Train the network on GPU

# For __init__, the GPT model parameters are:
#   - a token embedding table
#   - a positional embedding table
#   - a sequence of Blocks
#   - a layer norm
#   - a linear layer
#
# For forward(), the model consists in:
#   - applying the token embedding table and positional embedding table to the input tensor
#   - adding the two together
#   - applying the blocks, layer norm and linear layer (in that order)
#
# The code comes from hyperparameters that should work well on GPU.  On CPU, you 
# will need to reduce the model size significantly.
#
# In pytorch, an learnable embedding table is implemented with nn.Embedding(...)
#
# The token embedding table learns an embedding for each item of the vocabulary. The 
# positional embedding table does not depend on the input and learns an embedding
# for each position in the context.